## 1. Data Munging

### Preliminary setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import urllib
import json
import pandas as pd
import re

In [2]:
# Pandas config
pd.options.display.max_rows = 1600
pd.options.display.max_columns = 50

# Pyplot config
plt.rcParams.update({'font.size': 14})

In [3]:
# NHL season data
start_year = 1917 # 1917 is the first year with data
end_year = 2023
all_years = range(start_year, end_year, 1)
seasons = ['{}{}'.format(year, year+1) for year in all_years]
    
# Cup winners
#cup_winners = pd.read_csv('cup-winners.csv', names=['year', 'team', 'abbr'])
#cup_winners.loc[3] = ['2020', 'To Be Determined', 'TBD']
#cup_winners.index = seasons

In [4]:
req = urllib.request.urlopen('https://statsapi.web.nhl.com/api/v1/teams')
req_json = req.read().decode()
req_teams = dict(json.loads(req_json))['teams']
req_names = [team['name'] for team in req_teams]

In [5]:
# NHL Team Abbreviation Index

req = urllib.request.urlopen('https://statsapi.web.nhl.com/api/v1/teams')
req_json = req.read().decode()
req_teams = dict(json.loads(req_json))['teams']
req_names = [team['name'] for team in req_teams]

teams_long = ['Anaheim Ducks',
              'Arizona Coyotes',
              'Atlanta Flames',
              'Atlanta Thrashers',
              'Boston Bruins',
              'Brooklyn Americans',
              'Buffalo Sabres',
              'Calgary Flames',
              'California Golden Seals',
              'Carolina Hurricanes',
              'Chicago Blackhawks',
              'Cleveland Barons',
              'Colorado Avalanche',
              'Colorado Rockies',
              'Columbus Blue Jackets',
              'Dallas Stars',
              'Detroit Cougars',
              'Detroit Falcons',
              'Detroit Red Wings',
              'Edmonton Oilers',
              'Florida Panthers',
              'Hamilton Tigers',
              'Hartford Whalers',
              'Kansas City Scouts',
              'Los Angeles Kings',
              'Minnesota North Stars',
              'Minnesota Wild',
              'Montréal Canadiens',
              'Montreal Maroons',
              'Montreal Wanderers',
              'Nashville Predators',
              'New Jersey Devils',
              'New York Americans',
              'New York Islanders',
              'New York Rangers',
              'Oakland Seals',
              'Ottawa Senators',
              'Phoenix Coyotes',
              'Philadelphia Flyers',
              'Pittsburgh Penguins',
              'Pittsburgh Pirates',
              'Quebec Bulldogs',
              'Philadelphia Quakers',
              'Quebec Nordiques',
              'San Jose Sharks',
              'Seattle Kraken',
              'Ottawa Senators (1917)',
              'St. Louis Blues',
              'St. Louis Eagles',
              'Tampa Bay Lightning',
              'Toronto Arenas',
              'Toronto Maple Leafs',
              'Toronto St. Patricks',
              'Vancouver Canucks',
              'Vegas Golden Knights',
              'Washington Capitals',
              'Winnipeg Jets',
              'Winnipeg Jets (1979)']

teams_short = ['ANA',
               'ARI',
               'AFM',
               'ATL',
               'BOS',
               'BRK',
               'BUF',
               'CGY',
               'CGS',
               'CAR',
               'CHI',
               'CLE',
               'COL',
               'CLR',
               'CBJ',
               'DAL',
               'DCG',
               'DFL',
               'DET',
               'EDM',
               'FLA',
               'HAM',
               'HFD',
               'KCS',
               'LAK',
               'MNS',
               'MIN',
               'MTL',
               'MMR',
               'MWN',
               'NSH',
               'NJD',
               'NYA',
               'NYI',
               'NYR',
               'OAK',
               'OTT',
               'PHX',
               'PHI',
               'PIT',
               'PIR',
               'QBD',
               'QUA',
               'QUE',
               'SJS',
               'SEA',
               'SEN',
               'STL',
               'SLE',
               'TBL',
               'TAN',
               'TOR',
               'TSP',
               'VAN',
               'VGK',
               'WSH',
               'WPG', 
               'WIN']

team_index = dict(zip(teams_long, teams_short))

### Clean build for current season

In [6]:
### THIS IS A CLEAN BUILD FOR THE CURRENT SEASON, JUST A TEMPLATE ###

# First get the team data into a dataframe
teams_req = urllib.request.urlopen('https://statsapi.web.nhl.com/api/v1/teams?expand=team.stats')
teams_json = teams_req.read().decode()
teams_meta = dict(json.loads(teams_json))
teams_dict = {key['abbreviation']:key for key in teams_meta['teams']}
teams_df = pd.DataFrame.from_dict(teams_dict)

stats_dict = {team:teams_df[team]['teamStats'][0]['splits'][0]['stat'] for team in teams_df.columns}
stats_df = pd.DataFrame.from_dict(stats_dict, orient='index')

# Next we can get player rosters
players_req = urllib.request.urlopen('https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster')
players_json = players_req.read().decode()
players_meta = dict(json.loads(players_json))

### Pulling and building for all seasons

#### Team data

In [6]:
# Pull team data for all seasons from NHL API
teams_reqs = [urllib.request.urlopen( \
              'https://statsapi.web.nhl.com/api/v1/teams?expand=team.stats&season={}'.format( \
              season)) for season in seasons]

In [7]:
# Decode into readable JSON-like format
teams_jsons = [req.read().decode() for req in teams_reqs]

In [8]:
# Load in as JSON and collect appropriate segments into dictionary
teams_metas = [dict(json.loads(fread))['teams'] for fread in teams_jsons]

In [9]:
# Create list of all team data for each season
teams_dicts = [{key['abbreviation']:key for key in season} for season in teams_metas]

In [10]:
# Create master dictionary for all teams for each season
season_dict = dict(zip(seasons, teams_dicts))

In [11]:
# Create stats dictionary
stats_dict = {}

for key in season_dict:
    # Some seasons have no stats available, NaN results in KeyError
    try:
        stats_dict[key] = {season_dict[key][team]['abbreviation'] : \
                       season_dict[key][team]['teamStats'][0]['splits'][0]['stat'] \
                       for team in season_dict[key]}
    except KeyError:
    # Populate teams one at a time in years where some teams have no stats
        stats_dict[key] = {}
        for team in season_dict[key]:
            try:
                stats_dict[key][team] = season_dict[key][team]['teamStats'][0]['splits'][0]['stat']
            except KeyError:
                # Empty dictionary for teams with no stats (rather than NaN)
                stats_dict[key][team] = {key:0 for key in season_dict['20002001']['COL']['teamStats'][0]['splits'][0]['stat']}

In [15]:
# Create master dictionary of seasonal stats dataframes for each season
stats_dfs = {key:pd.DataFrame.from_dict(stats_dict[key], orient='index') for key in stats_dict}

In [18]:
# Re-orient stats_dfs by team with [team][season] as keys
team_data_dict = {}

for team in teams_short:
    inner_season_dict = {}
    
    for season in season_dict.keys():
        try:
            inner_season_dict[season] = stats_dict[season][team]
        except:
            inner_season_dict[season] = {}

    team_data_dict[team] = inner_season_dict

In [44]:
# Convert team data to master dataframes dictionary
team_data_dfs = {key:pd.DataFrame.from_dict(team_data_dict[key], orient='index') for key in team_data_dict}

In [45]:
# Convert to Series for concatenation
to_send = pd.Series(team_data_dfs)

In [46]:
# Add the season and team_id columns
for key, df in to_send.items():
    df['season'] = df.index
    df['team_id'] = key

In [48]:
# Concatenate into a single data structure to send to SQL
combined_df = pd.concat(to_send.values, ignore_index=False)

In [50]:
# Migrate to Azure postgres database
from sqlalchemy import create_engine
from sqlalchemy.sql import text

# Replace the following with your Azure PostgreSQL connection details
database_username = "adampaul"
database_password = "pass"
database_host = "compendia.postgres.database.azure.com"
database_port = "5432"
database_name = "compendia_db"

# Create the connection string
conn_str = f"postgresql://{database_username}:{database_password}@{database_host}:{database_port}/{database_name}"

# Create the database engine
engine = create_engine(conn_str)

# Write the combined DataFrame to the database
table_name = "team_data"
combined_df.to_sql(table_name, engine, if_exists="replace", index=False)

# Add the composite primary key
with engine.connect() as connection:
    connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY (team_id, season);"))

In [15]:
# Append a new boolean array column to each seasonal dataframe to indicate Stanley Cup winner
#for key in stats_dfs:
#    win_bool = stats_dfs[key].index == cup_winners.loc[key]['abbr']
#    stats_dfs[key]['cupWin'] = win_bool

#### Player data (via ID lookup)

In [6]:
# Pull the roster data? 
player_reqs = [urllib.request.urlopen( \
              'https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster&season={}'.format( \
              season)) for season in seasons]

In [7]:
# Decode into readable JSON-like format
player_jsons = [req.read().decode() for req in player_reqs]

In [8]:
# Load in as JSON and collect appropriate segments into dictionary
player_metas = [dict(json.loads(fread))['teams'] for fread in player_jsons]

In [9]:
# Create a dictionary that connects each season to each team that played that season, and connect each of those teams to their corresponding roster data
rosters = []

for season in player_metas:
    try:
        rosters.append({team['abbreviation']:team['roster']['roster'] for team in season})
        
    except KeyError:
        team_dict = {}
        for team in season:
            try:
                team_dict[team['abbreviation']] = team['roster']['roster']
            except KeyError:
                team_dict[team['abbreviation']] = {}
        rosters.append(team_dict)

season_dicts = dict(zip(seasons, rosters))

In [22]:
# Gather all of the player IDs and their full names
all_ids = [season_dicts[season][team][x]['person']['id'] for season in seasons for team in season_dicts[season].keys() for x in range(len(season_dicts[season][team]))]
all_names = [season_dicts[season][team][x]['person']['fullName'] for season in seasons for team in season_dicts[season].keys() for x in range(len(season_dicts[season][team]))]

# Then combine them into a dictionary
id_dict = dict(zip(all_names, all_ids))

In [25]:
# Create a function that returns a player's year-by-year statistics

def get_player_df(player):
    # Make API request and parse into python data structure
    stat_req = urllib.request.urlopen('https://statsapi.web.nhl.com/api/v1/people/{}/stats?stats=yearByYear'.format(id_dict[player]))
    stat_json = stat_req.read().decode()
    player_stat = dict(json.loads(stat_json))['stats'][0]['splits']
    
    # Convert to pandas dataframe with team column
    player_stats = [player_season['stat'] for player_season in player_stat if player_season['league']['name'] == 'National Hockey League']
    player_seasons = [player_season['season'] for player_season in player_stat if player_season['league']['name'] == 'National Hockey League']
    player_teams = [player_season['team']['name'] for player_season in player_stat if player_season['league']['name'] == 'National Hockey League']
    teams_abb = [team_index[team] for team in player_teams]
    player_df = pd.DataFrame(player_stats, index=player_seasons)
    player_df.insert(loc=0, column='team', value=teams_abb)
    
    return player_df

In [26]:
player_df = get_player_df('Jaromir Jagr')

In [27]:
player_dfs = {player:get_player_df(player) for player in id_dict.keys()}

In [38]:
player_dfs['Brett Hull']

,team,assists,goals,pim,shots,games,powerPlayGoals,powerPlayPoints,penaltyMinutes,shotPct,gameWinningGoals,overTimeGoals,shortHandedGoals,shortHandedPoints,plusMinus,points,timeOnIce,hits,powerPlayTimeOnIce,evenTimeOnIce,faceOffPct,shortHandedTimeOnIce,blocked,shifts
19861987,CGY,0,1,0,5,5,0,0,0,20.00,1,0,0,0,-1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19871988,CGY,24,26,12,153,52,4,13,12,16.99,3,0,0,0,10,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19871988,STL,8,6,4,58,13,2,3,4,10.34,0,0,0,0,4,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19881989,STL,43,41,33,305,78,16,42,33,13.44,6,1,0,0,-17,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19891990,STL,41,72,24,385,80,27,44,24,18.70,12,0,0,0,-1,113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19901991,STL,45,86,22,389,78,29,45,22,22.11,11,1,0,0,23,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19911992,STL,39,70,48,408,73,20,40,48,17.16,9,1,5,5,-2,109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19921993,STL,47,54,41,390,80,29,58,41,13.85,2,0,0,0,-27,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19931994,STL,40,57,38,392,81,25,41,38,14.54,6,0,3,5,-3,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19941995,STL,21,29,10,200,48,9,16,10,14.50,6,1,3,6,13,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
to_db = pd.Series(player_dfs)

In [40]:
for key, df in to_db.items():
    df['player_name'] = key
    df['season'] = df.index

In [42]:
combined_player_df = pd.concat(to_db.values, ignore_index=True)

#### END

In [43]:
def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

# Rename columns
combined_player_df.columns = [camel_to_snake(col) for col in combined_player_df.columns]

In [44]:
columns_to_replace = ['penalty_minutes', 'time_on_ice', 'power_play_time_on_ice', 'even_time_on_ice', 'short_handed_time_on_ice']

for col in columns_to_replace:
    combined_player_df.loc[combined_player_df[col].isna(), col] = pd.NaT

In [45]:
# Identify rows with duplicated values in columns 'A', 'B', and 'C'
duplicated_rows = combined_player_df.duplicated(subset=['season', 'player_name', 'team'], keep=False)

# Display the duplicated rows
doubles = combined_player_df[duplicated_rows]['player_name']

In [47]:
double_traded = ['George Carroll',
                 'Roy Worters',
                 'Ron Lyons',
                 'Claude Bourque',
                 'Paul Masnick',
                 'Larry Brown',
                 'Gary Mcadam',
                 'Vaclav Nedomansky',
                 'Dean Kennedy',
                 'Jarrod Skalde',
                 'Harold Druken',
                 'Craig Anderson',
                 'Joel Perrault',
                 'David Koci',
                 'Ben Maxwell',
                 'Derek Grant',
                 'Chris Wagner']

In [48]:
from datetime import datetime, timedelta

def custom_time_parser(time_str):
    if ':' not in time_str:
        return None
    parts = time_str.split(':')
    if len(parts) == 3:
        hours, minutes, seconds = [int(p) for p in parts]
    elif len(parts) == 2:
        hours = 0
        minutes, seconds = [int(p) for p in parts]
    else:
        return None
    
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return timedelta(seconds=total_seconds)

def sum_time_objects(series):
    time_objects = [custom_time_parser(t) if not pd.isna(t) else t for t in series]
    total_seconds = sum([t.total_seconds() for t in time_objects if not pd.isna(t)])
    total_seconds = int(total_seconds)
    minutes, seconds = divmod(total_seconds, 60)
    return f"{minutes:02d}:{seconds:02d}"

def sum_pims(series):
    total_minutes = sum(int(t) for t in series if not pd.isna(t))
    return total_minutes

aggregations = {
    'team': 'first',
    'player_name': 'first',
    'season': 'first',
    'assists': 'sum',
    'goals': 'sum',
    'pim': 'sum',
    'games': 'sum',
    'game_winning_goals': 'sum',
    'over_time_goals': 'sum',
    'points': 'sum',
    'shutouts': 'sum',
    'ties': 'sum',
    'wins': 'sum',
    'losses': 'sum',
    'goal_against_average': 'sum',
    'games_started': 'sum',
    'goals_against': 'sum',
    'power_play_goals': 'sum',
    'power_play_points': 'sum',
    'short_handed_goals': 'sum',
    'short_handed_points': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'face_off_pct': 'sum',
    'shot_pct': 'sum',
    'blocked': 'sum',
    'plus_minus': 'sum',
    'shifts': 'sum',
    'saves': 'sum',
    'power_play_saves': 'sum',
    'short_handed_saves': 'sum',
    'even_saves': 'sum',
    'save_percentage': 'sum',
    'shots_against': 'sum',
    'short_handed_shots': 'sum',
    'even_shots': 'sum',
    'power_play_shots': 'sum',
    'power_play_save_percentage': 'sum',
    'short_handed_save_percentage': 'sum',
    'even_strength_save_percentage': 'sum',
    'ot': 'sum',

    # Time columns
    'penalty_minutes': lambda x: sum_pims(x),
    'time_on_ice': lambda x: sum_time_objects(x),
    'power_play_time_on_ice': lambda x: sum_time_objects(x),
    'even_time_on_ice': lambda x: sum_time_objects(x),
    'short_handed_time_on_ice': lambda x: sum_time_objects(x),
}

# Step 2: Filter the DataFrame
filtered_df = combined_player_df[combined_player_df['player_name'].isin(double_traded)]

columns_to_replace = ['penalty_minutes', 'time_on_ice', 'power_play_time_on_ice', 'even_time_on_ice', 'short_handed_time_on_ice']

for col in columns_to_replace:
    filtered_df.loc[filtered_df[col].isna(), col] = pd.NaT

'''
def convert_to_time(x):
    try:
        return datetime.strptime(x, "%M:%S").time()
    except:
        return x

for col in columns_to_replace:
    filtered_df[col] = filtered_df[col].apply(convert_to_time)
'''

'\ndef convert_to_time(x):\n    try:\n        return datetime.strptime(x, "%M:%S").time()\n    except:\n        return x\n\nfor col in columns_to_replace:\n    filtered_df[col] = filtered_df[col].apply(convert_to_time)\n'

In [49]:
# Step 3: Group and aggregate
grouped_df = filtered_df.groupby(['team', 'player_name', 'season']).agg(aggregations).reset_index(drop=True)

In [50]:
# Step 4: Merge the aggregated rows back to the original DataFrame
combined_player_df = combined_player_df[~combined_player_df['player_name'].isin(double_traded)].append(grouped_df).reset_index(drop=True)

/tmp/ipykernel_29941/2675136325.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_player_df = combined_player_df[~combined_player_df['player_name'].isin(double_traded)].append(grouped_df).reset_index(drop=True)


In [51]:
combined_player_df['player_name']

0        George Boucher
1        George Boucher
2        George Boucher
3        George Boucher
4        George Boucher
              ...      
53563      Dean Kennedy
53564      Dean Kennedy
53565       Ben Maxwell
53566    Craig Anderson
53567       Gary Mcadam
Name: player_name, Length: 53568, dtype: object

In [52]:
final_player_df = combined_player_df.copy()

In [55]:
final_player_df['season']

0        19171918
1        19181919
2        19191920
3        19201921
4        19211922
           ...   
53563    19921993
53564    19931994
53565    20112012
53566    20202021
53567    19831984
Name: season, Length: 53568, dtype: object

In [56]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

# Replace the following with your Azure PostgreSQL connection details
database_username = "adampaul"
database_password = "pass"
database_host = "compendia.postgres.database.azure.com"
database_port = "5432"
database_name = "compendia_db"

# Create the connection string
conn_str = f"postgresql://{database_username}:{database_password}@{database_host}:{database_port}/{database_name}"

# Create the database engine
engine = create_engine(conn_str)

# Write the combined DataFrame to the database
table_name = "player_data_new"
final_player_df.to_sql(table_name, engine, if_exists="replace", index=False)

318